In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
from newspaper import Article
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [156]:
# data directory
data_dir = Path("../data")

input_dir = data_dir / "input"
raw_dir = data_dir / "raw"
output_dir = data_dir / "processed"
sample_dir = data_dir / "samples"

rss_dir = raw_dir / "rss"
twitter_dir = raw_dir / "twitter"

## Load all URLs

In [3]:
rss_urls = pd.read_csv(raw_dir / "cleaned_rss.csv")
tweet_urls = pd.read_csv(raw_dir / "cleaned_tweets.csv")

In [4]:
urls = rss_urls[["source", "channel", "url"]]
urls = urls.append(tweet_urls[["source", "channel", "url"]])
# urls["url_section"] = None
urls = urls.reset_index(drop=True)
urls.index.name = "url_id"
urls

,source,channel,url
url_id,,,
0,medpage,rss,https://www.medpagetoday.com/infectiousdisease...
1,medpage,rss,https://www.medpagetoday.com/special-reports/e...
2,medpage,rss,https://www.medpagetoday.com/infectiousdisease...
3,medpage,rss,https://www.medpagetoday.com/pulmonology/gener...
4,medpage,rss,https://www.medpagetoday.com/infectiousdisease...
...,...,...,...
7120,popsci,twitter,https://www.popsci.com/story/diy/how-to-use-an...
7121,popsci,twitter,https://www.popsci.com/how-to-make-perfect-pie...
7122,popsci,twitter,https://www.popsci.com/story/technology/airbus...


## Download and parse articles

In [108]:
def get_published(article, source):
    published = ""
    if source in ["guardian", "nyt", "newsmed", "healthday", "popsci"]:
        published = article.meta_data["article"]["published_time"]
    elif source in ["medpage"]:
        published = article.meta_data["dc.date"]
    elif source == "ifls":
        soup = BeautifulSoup(article.html, 'lxml')
        published = soup.find_all("span", class_="date")[0].a.string
    elif source == "wired":
        soup = BeautifulSoup(article.html, 'lxml')
        published = soup.find_all("time", attrs={"data-testid":"ContentHeaderPublishDate"})[0].string
    return pd.to_datetime(published).isoformat()

def get_modified(article, source):
    modified = ""
    if source in ["guardian", "nyt", "newsmed", "healthday", "popsci"]:
        modified = article.meta_data["article"]["modified_time"]
    return pd.to_datetime(modified).isoformat()

def get_section(article, source):
    section = ""
    if source in ["guardian", "nyt", "healthday"]:
        section = article.meta_data["article"]["section"]
    elif source == "popsci":
        soup = BeautifulSoup(article.html, 'lxml')
        section = soup.find_all("ul", class_="article-categories")[0].find_all("a")[0].string
    elif source == "wired":
        soup = BeautifulSoup(article.html, 'lxml')
        section = soup.find_all("a", class_="rubric__link")[0].span.string
    elif source == "ifls":
        soup = BeautifulSoup(article.html, 'lxml')
        section = soup.find_all("a", class_="category")[0].string
    elif source == "medpage":
        section = article.meta_data["sailthru.topcat"]
    elif source == "newsmed":
        soup = BeautifulSoup(article.html, 'lxml')
        section = soup.find_all(class_="active-site-sections-menu-btn")[0].string
        section = section.split(" Home")[0]
        
    if len(section) == 0:
        section = ""
    return section.strip()

def get_keywords(article, source):
    keywords = ""
    if source in ["guardian", "nyt", "wired", "newsmed", "medpage"]:
        if "news_keywords" in article.meta_data:
            keywords = article.meta_data["news_keywords"]
        elif "keywords" in article.meta_data:
            keywords = article.meta_data["keywords"]
        elif "tag" in article.meta_data["article"]:
            keywords = article.meta_data["article"]["tag"]
    elif source == "popsci":
        soup = BeautifulSoup(article.html, 'lxml')
        keywords = soup.find_all("div", class_="tag-item")
        keywords = [tag.a.string.strip() for tag in keywords]
        keywords = ", ".join(keywords)
    elif source in ["ifls", "healthday"]:
        article.nlp()
        keywords = ", ".join(article.keywords)
    if len(keywords) == 0:
        keywords = ""
    return keywords
    
def get_authors(article, source):
    authors = ""
    if source in ["guardian", "nyt", "popsci"]:
        authors = ", ".join(article.authors)
    elif source == "wired":
        authors = article.meta_data["author"]
    elif source == "ifls":
        soup = BeautifulSoup(article.html, 'lxml')
        authors = soup.find_all("div", class_="author")[0]
        authors = authors.find_all(class_="name")
        if len(authors) > 0:
            authors = authors[0].a.string
            authors = authors.split("By ")[-1]
        else:
            authors = ""
    elif source == "newsmed":
        soup = BeautifulSoup(article.html, 'lxml')
        authors = soup.find_all("span", attrs={"itemprop":"name"})
        if len(authors) > 0:
            authors = authors[0].string
        else:
            authors = ""
    elif source == "medpage":
        authors = article.meta_data["sailthru.author"]
    elif source == "healthday":
        soup = BeautifulSoup(article.html, 'lxml')
        authors = soup.find_all("a", class_="post-author__name")[0].string
        
    if len(authors) == 0:
        authors = ""
    return authors

In [130]:
domains = {
    "guardian": "theguardian",
    "healthday": "healthday",
    "ifls": "iflscience",
    "medpage": "medpagetoday",
    "newsmed": "news-med",
    "nyt": "nytimes",
    "popsci": "popsci",
    "wired": "wired"
}

def process_url(url, source):
    # Init metadata
    meta = {
        "published": "",
        "modified": "",
        "section": "",
        "keywords": "",
        "authors": "",
        "title": "",
        "canonical_url": "",
        "success": True,
        "error_msg": ""
    }
        
    # Attempt to parse content
    try:
        article = Article(url)
        article.download()
        article.parse()
    except Exception as e:
        meta["success"] = False
        meta["error_msg"] = str(e)
        return meta
    
    # use canonical link to determine if link is not from source
    if domains[source] not in article.canonical_link:
        meta["success"] = False
        meta["error_msg"] = "URL with wrong domain"
        return meta

    meta["title"] = article.title
    meta["canonical_url"] = article.canonical_link
    
    try:
        meta["published"] = get_published(article, source)
    except:
        pass
    
    try:
        meta["modified"] = get_modified(article, source)
    except:
        pass
    
    try:
        meta["section"] = get_section(article, source)
    except:
        pass
    
    try:
        meta["keywords"] = get_keywords(article, source)
    except:
        pass
    
    try:
        meta["authors"] = get_authors(article, source)
    except:
        pass
    
    return meta

In [131]:
outcols = [
    "published",
    "modified",
    "section",
    "keywords",
    "authors",
    "title",
    "canonical_url",
    "success",
    "msg",
]
outcols = list(urls.columns) + outcols
    
outfile = output_dir / "articles.csv"
if outfile.exists():
    outputs = pd.read_csv(outfile, index_col="url_id")
else:
    outputs = urls.copy()
    outputs = outputs.reindex(columns=outcols, fill_value=None)

In [132]:
df = missing_urls = outputs[outputs.success != True]
df = df.sample(frac=1)
total = len(df)

for ix, row in tqdm(df.iterrows(), total=total):
    try:
        meta = process_url(row.url, row.source)
    except Exception as e:
        print(row.url, row.source)
        print(traceback.format_exc())
        continue

    for k, v in meta.items():
        outputs.loc[ix, k] = v

  0%|          | 0/172 [00:00<?, ?it/s]

In [142]:
outputs = outputs.replace("", None)
outputs.to_csv(outfile)

In [158]:
# sample = outputs[outputs.success].groupby("source", as_index=False).apply(lambda x: x.sample(10)).reset_index(drop=True)
# sample.index.name = "id"
# sample.to_csv(sample_dir / "all_venues_sample_05052021.csv")